In [1]:
%matplotlib inline

import base64
import io
import glob
import os
import random

from IPython import display
from IPython.display import HTML
import matplotlib.pyplot as plt
import numpy as np

import torch

from gym.wrappers import Monitor
from pyvirtualdisplay import Display

import sys
sys.path.append('../src')

from common.atari_wrappers import make_atari
from nets import DQN
from utils import wrap_atari_dqn

In [2]:
vdisplay = Display(visible=0, size=(1400, 900))
display_params = vdisplay.start()

In [15]:
def get_state(obs, args=None):
    obs = np.asarray(obs)
    obs = obs.transpose(2, 0, 1)  # -> (c, h, w)
    obs = obs / 255.
    state = torch.from_numpy(obs).type(torch.float).unsqueeze(0)
    return state


def show_video(path):
    pass


def show_video(logdir):
  mp4list = glob.glob(os.path.join(logdir, 'video/*.mp4'))
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    # mp4 = mp4list[-1]
    print(mp4)
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    display.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")

In [4]:
logdir = '../logs'
expid = '20200724121347'
# checkpoint_name = 'checkpoint-000027500-14.0.pt'
checkpoint_name = 'checkpoint-000028400-13.0.pt'
checkpoint_path = os.path.join(logdir, expid, checkpoint_name)

env_name = 'BreakoutNoFrameskip-v4'
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

epsilon = -1

In [5]:
env = make_atari(env_name)
env = wrap_atari_dqn(env)
_path = os.path.join(logdir, expid, 'video')

In [6]:
checkpoint = torch.load(checkpoint_path)
policy_net = DQN(84, 84, env.unwrapped.action_space.n)
policy_net.load_state_dict(checkpoint['policy_net'])
policy_net.to(device)
policy_net.eval()

DQN(
  (conv1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
  (conv2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=3136, out_features=512, bias=True)
  (out): Linear(in_features=512, out_features=4, bias=True)
)

In [7]:
def validate(env, model, epsilon):
    obs = env.reset()
    return 

In [8]:
def get_action(state, model, epsilon):
    r = random.random()
    if r > epsilon:
        action = model(state).max(dim=1)[1]
        action = action.view(1, 1)
    else:
        action = torch.tensor([[random.randrange(env.action_space.n)]], device=device, dtype=torch.long)
    return action

In [9]:
env = Monitor(env, _path, force=True)
obs = env.reset()
state = get_state(obs).to(device)
G = 0
while True:
    env.render(mode='rgb_array')
    action = get_action(state, policy_net, epsilon)
    obs, reward, done, info = env.step(action.item())
    G += reward
    next_state = get_state(obs).to(device)
    if done:
        break
    else:
        state = next_state
        
print('EpisodeReward {}'.format(G))

EpisodeReward 33.0


In [10]:
obs = env.reset()
state = get_state(obs).to(device)
G = 0
while True:
    env.render(mode='rgb_array')
    action = get_action(state, policy_net, epsilon)
    obs, reward, done, info = env.step(action.item())
    G += reward
    next_state = get_state(obs).to(device)
    if done:
        break
    else:
        state = next_state
        
print('EpisodeReward {}'.format(G))

EpisodeReward 6.0


In [11]:
!ls ../logs/20200724121347/video

openaigym.video.0.4835.video000000.meta.json
openaigym.video.0.4835.video000000.mp4
openaigym.video.0.4835.video000001.meta.json
openaigym.video.0.4835.video000001.mp4


In [16]:
show_video(os.path.join(logdir, expid))

../logs/20200724121347/video/openaigym.video.0.4835.video000000.mp4
